In [1]:
using DataFrames, TableOperations, Tables, Random
using Recommender: Movielens100k, load_dataset, ratio_split, ImplicitMF, evaluate_u2i, PrecisionAtK, RecallAtK, NDCG, fit!, predict_u2i, make_u2i_dataset

┌ Info: Precompiling Recommender [f1dcbcfb-0b1d-45c0-883e-2edcfc1d4c23]
└ @ Base loading.jl:1317


In [2]:
ml100k = Movielens100k()
download(ml100k)
rating, user, movie = load_dataset(ml100k);

In [3]:
rating = rating |> TableOperations.filter(x->Tables.getcolumn(x, :rating) >= 4)

TableOperations.Filter{var"#1#2", CSV.File{false}}(var"#1#2"(), CSV.File("/Users/keisuke.yanagi/workspace/Recommender.jl/src/dataset/../../dataset/movielens100k/u.data"):
Size: 100000 x 4
Tables.Schema:
 :userid     Int64
 :movieid    Int64
 :rating     Int64
 :timestamp  Int64)

In [4]:
movie2title = Dict()
for row in Tables.rows(movie)
    movie2title[row[:movieid]] = row[:movie_title]
end
rating = rating |> TableOperations.transform(Dict(:movieid=>x->movie2title[x]))

TableOperations.Transforms{false, TableOperations.Filter{var"#1#2", CSV.File{false}}, Dict{Symbol, var"#3#4"}}(TableOperations.Filter{var"#1#2", CSV.File{false}}(var"#1#2"(), CSV.File("/Users/keisuke.yanagi/workspace/Recommender.jl/src/dataset/../../dataset/movielens100k/u.data"):
Size: 100000 x 4
Tables.Schema:
 :userid     Int64
 :movieid    Int64
 :rating     Int64
 :timestamp  Int64), Dict(:movieid => var"#3#4"()))

In [5]:
Random.seed!(1234);
train_valid_table, test_table = ratio_split(rating, 0.8)

train_table, valid_table = ratio_split(train_valid_table, 0.8)
length(Tables.rows(train_table)), length(Tables.rows(valid_table)), length(Tables.rows(test_table))

(35440, 8860, 11075)

In [6]:
prec10 = PrecisionAtK(10)
recall10 = RecallAtK(10)
ndcg10 = NDCG(10)
metrics = [prec10, recall10, ndcg10]

3-element Vector{Recommender.AbstractMetric}:
 PrecisionAtK(10)
 RecallAtK(10)
 NDCG(10)

In [7]:
model = ImplicitMF(128, true, 0.005)

ImplicitMF(128, Recommender.Logloss(), true, 0.005, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing)

In [8]:
evaluate_u2i(model, train_valid_table, test_table, metrics, 10, col_item=:movieid, n_epochs=256, n_negatives=8, learning_rate=0.002, drop_history=true)

┌ Info: epoch=1: train_loss=Inf, val_loss=0.0
└ @ Recommender /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/implicit_mf.jl:144
┌ Info: epoch=2: train_loss=0.4215107029571365, val_loss=0.0
└ @ Recommender /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/implicit_mf.jl:144
┌ Info: epoch=3: train_loss=0.3959135706449421, val_loss=0.0
└ @ Recommender /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/implicit_mf.jl:144
┌ Info: epoch=4: train_loss=0.3759857227126211, val_loss=0.0
└ @ Recommender /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/implicit_mf.jl:144
┌ Info: epoch=5: train_loss=0.3615996320258849, val_loss=0.0
└ @ Recommender /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/implicit_mf.jl:144
┌ Info: epoch=6: train_loss=0.34986899066065563, val_loss=0.0
└ @ Recommender /Users/keisuke.yanagi/workspace/Recommender.jl/src/model/implicit_mf.jl:144
┌ Info: epoch=7: train_loss=0.339559362199621, val_loss=0.0
└ @ Recommender /Users/keisuke.yanag

(NDCG(10) = 0.24531654370701705, RecallAtK(10) = 0.19573552356097224, PrecisionAtK(10) = 0.18119565217391229)